返回dfoff每个userid中，【只领券，核销数，直接买，单商铺最大购买次数，成交商家数】的数量

office 2 2368.7907860279083s  
office 2 2276.1418340206146 s  
home  1  2320  
home  2  1503.2748665809631 s  
home  2  1475.3602182865143 s  
home  6  1271.9127905368805 s  
home  8  1239.525927066803 s  
home 12  1383.0938215255737 s  

In [1]:
import sys 
sys.path.append('../app/')
import fileio
import preprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import time
import threading

In [2]:
# 读取数据
dfoff ,dftest,dfon = fileio.read_all()
# 特征处理
dfoff = preprocess.get_label(dfoff)
dfoff = preprocess.process_manjian(dfoff)


#这次使用未泄露的数据,
dfoff,_ = preprocess.split_receivetime(dfoff)

user_off_list = dfoff.User_id.unique()

In [5]:
def user_features(uid):
    """返回userid，只领券，核销数，直接买，单商铺最大购买次数，成交商家数"""
    lcdict = {0:0,1:0,-1:0}
    df001 = dfoff[dfoff.User_id==uid]
    
    dflabelcount = df001.label.value_counts()
    dfMerchantcount = df001.Merchant_id.value_counts()
    for i in dflabelcount.index:
        lcdict[i] = dflabelcount[i]
    
    return uid,lcdict[0],lcdict[1],lcdict[-1],dfMerchantcount.max(),dfMerchantcount.count()

class myThread(threading.Thread):
    def __init__(self,threadID,name,user_off_list):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.user_off_list = user_off_list

        
    def run(self):
        print("开始线程"+self.name)
        self.obj = test_fun(self.user_off_list,self.name)
        print("退出线程"+self.name)
        
def test_fun(user_off_list,name):
    n = 0.
    ll = []
    for uid in  user_off_list:
        if n%1000==0:
            print("{0}---{1}%".format(name,n*100/len(user_off_list)) )
        ll.append(user_features(uid))
        n = n+1
    return ll

In [6]:

# 创建新线程
halfnum = int(len(user_off_list)/2)

#def earthworm(core_num=2):
#    user
thread1 = myThread(1, "Thread-1",user_off_list[0:halfnum])
thread2 = myThread(2, "Thread-2",user_off_list[halfnum:])


time_start=time.time()
# 开启新线程
thread1.start()
thread2.start()
thread1.join()
thread2.join()
print ("退出主线程")

time_end=time.time()
print('time cost',time_end-time_start,'s')

开始线程Thread-1
Thread-1---0.0%
开始线程Thread-2
Thread-2---0.0%
Thread-1---0.48334630312580057%
Thread-2---0.48334630312580057%
Thread-1---0.9666926062516011%
Thread-2---0.9666926062516011%
Thread-1---1.4500389093774015%
Thread-2---1.4500389093774015%
Thread-1---1.9333852125032023%
Thread-2---1.9333852125032023%
Thread-1---2.4167315156290026%
Thread-2---2.4167315156290026%
Thread-2---2.900077818754803%Thread-1---2.900077818754803%

Thread-1---3.3834241218806036%
Thread-2---3.3834241218806036%
Thread-1---3.8667704250064046%
Thread-2---3.8667704250064046%
Thread-2---4.350116728132205%
Thread-1---4.350116728132205%
Thread-2---4.833463031258005%
Thread-1---4.833463031258005%
Thread-2---5.316809334383806%
Thread-1---5.316809334383806%
Thread-2---5.800155637509606%
Thread-1---5.800155637509606%
Thread-2---6.283501940635407%
Thread-1---6.283501940635407%
Thread-2---6.766848243761207%
Thread-1---6.766848243761207%
Thread-2---7.250194546887008%
Thread-1---7.250194546887008%
Thread-2---7.7335408500128

Thread-1---65.25175092198307%
Thread-2---65.25175092198307%
Thread-1---65.73509722510887%
Thread-2---65.73509722510887%
Thread-1---66.21844352823467%
Thread-2---66.21844352823467%
Thread-1---66.70178983136047%
Thread-2---66.70178983136047%
Thread-1---67.18513613448627%
Thread-2---67.18513613448627%
Thread-1---67.66848243761207%
Thread-2---67.66848243761207%
Thread-1---68.15182874073788%
Thread-2---68.15182874073788%
Thread-1---68.63517504386368%
Thread-2---68.63517504386368%
Thread-1---69.11852134698948%
Thread-2---69.11852134698948%
Thread-1---69.60186765011528%
Thread-2---69.60186765011528%
Thread-1---70.08521395324108%
Thread-2---70.08521395324108%
Thread-1---70.56856025636688%
Thread-2---70.56856025636688%
Thread-1---71.05190655949268%
Thread-2---71.05190655949268%
Thread-1---71.53525286261848%
Thread-2---71.53525286261848%
Thread-1---72.01859916574428%
Thread-2---72.01859916574428%
Thread-1---72.50194546887008%
Thread-2---72.50194546887008%
Thread-1---72.98529177199588%
Thread-2--

In [7]:
dflist = [pd.DataFrame(dfs) for dfs in [thread1.obj,thread2.obj]]
dfobj = pd.concat(dflist)
dfobj.index = range(0,len(dfobj))
dfobj.columns = ['用户id','只领券','核销数','直接买','单商铺最大购买次数','成交商家数']

In [8]:
dfobj.head()

,用户id,只领券,核销数,直接买,单商铺最大购买次数,成交商家数
0,1439408,2,0,0,2,1
1,1832624,1,0,0,1,1
2,2029232,1,0,0,1,1
3,2223968,1,0,0,1,1
4,73611,1,0,0,1,1


In [9]:
path = '../storage/newfeatures/dfoff_add_f1/'
fileio.ifmakdir(path)

#dfobj.to_csv(path+"new_fea_1",index = 0)
dfobj.to_csv(path+"new_fea_1_offdeceived.csv",index = 0)